<a href="https://colab.research.google.com/github/EvanWAppel/work-examples/blob/main/Alt_Coin_Aggregator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Dependencies
import os
import pandas as pd
%load_ext google.colab.data_table
from google.colab import data_table
import numpy as np
from google.colab import files
import datetime as datetime
import plotly.express as px
# For dealing with big tables
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 800)

In [ ]:
# Import Base Data Frames
data = pd.read_csv("trimmed_orders.csv", parse_dates=True)
vis = pd.read_csv("visionData (11).csv", parse_dates=True)
print(vis.head())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,28,38,39,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   Transaction Id  Customer Id   Machine Id Kind        Status Location Id State  Amount Deposited USD  Actual Amount BTC  Amount Deposited BTC  Requested Amount USD  Actual Amount USD  Amount to be Refunded  Alt Coins Amount Alt Coins Currency  Spot Price  Transaction Price Promo Code  Margin Percentage                              Wallet Address                                      Blockchain ID  Network Fees  Mining Fees BTC           Cash Timestamp Sell Timestamp       Complete Timestamp  Gross Revenue  Transaction Gross Profit  Mining Fee USD  Machine Gross Profit  Refund Amount USD  Refund Amount BTC  KYC Tier
0  161072909163bc  17024459814  Vision01_HQ  BUY      COMPLETE       00000    NV                 100.0           0.002788                   NaN                   NaN                NaN                    NaN               NaN                NaN         NaN          35873.320        NaN                NaN          1646yseYmhkzVGxCEHhPoP3yFNXSjagvNr  00000000000000000003e4850

In [ ]:
data = data[["ID","Short.Code","Key","Customer.ID","BTM.Machine.Name","Kind","State","Amount.Deposited","Deposit.Currency","Requested.Withdrawal.Amount","Actual.Withdrawal.Amount","Withdrawal.Currency","Withdrawal.Address","Price","Spot.Price","Margin.Percentage","Blockchain.ID","CloudWallet.Network.Fee","Key.Org","Key.Product","KYC.Level","KYC.Limit.Deposit","KYC.Limit.Withdraw","KYC.Limit.Txns","Bitaccess.Fee","Trade.ID","Trade.Exchange.Name","Trade.Type","Trade.Status","Trade.Price","Trade.Amount.Out","Trade.Amount.In","Trade.Details.Fee","Duration","Created.At","Reference.Order","BTM.Reference.Transaction.ID","Mining.Fee","Location.ID","Promo.Code","Promo.Discount","Altcoin.selected","Altcoin.address","Completed.At","Closed.At"]]
data2 = data[['ID','Customer.ID','BTM.Machine.Name','Kind','State','Amount.Deposited','Deposit.Currency','Actual.Withdrawal.Amount','Withdrawal.Currency','Margin.Percentage','KYC.Level','Created.At','Location.ID','Promo.Code']]
data2 = data2[(data2['Created.At']>='2017')&(data2['Created.At']<'2021')]
data3 = data[(data['Completed.At']>='2021')&(data['Completed.At']<'2022')]
data3 = data3[data3['Altcoin.selected'].notnull()]
data3 = data3[(data3['State']=='completed')]
data3 = data3[['Kind','Amount.Deposited','Deposit.Currency','Actual.Withdrawal.Amount','Withdrawal.Currency','Altcoin.selected','Created.At']]
basell = data3[data3['Kind']=='btmsell']
basell = basell.groupby(['Altcoin.selected','Created.At'],as_index=False).agg({'Actual.Withdrawal.Amount':sum, 'Withdrawal.Currency':np.size})
basell = basell.rename(columns={'Altcoin.selected':'currency','Actual.Withdrawal.Amount':'dollars','Withdrawal.Currency':'count','Created.At':'date'})
basell = basell.assign(type='sell')
basell = basell.assign(os='ba')
babuy = data3[data3['Kind']=='btmbuy']
babuy = babuy.groupby(['Altcoin.selected','Created.At'],as_index=False).agg({'Amount.Deposited':sum,'Deposit.Currency':np.size})
babuy = babuy.rename(columns={'Altcoin.selected':'currency','Amount.Deposited':'dollars','Deposit.Currency':'count','Created.At':'date'})
babuy = babuy.assign(type='buy')
babuy = babuy.assign(os='ba')
print(basell.head())

  currency                 date  dollars  count  type  os
0      ADA  2021-08-09 19:19:48    900.0      1  sell  ba
1      ADA  2021-08-14 17:40:37    600.0      1  sell  ba
2      ADA  2021-08-28 15:48:39    180.0      1  sell  ba
3      ADA  2021-09-08 17:04:07    180.0      1  sell  ba
4      ADA  2021-09-09 02:27:23    200.0      1  sell  ba


In [ ]:
visdata = vis[(vis['Status']=='COMPLETE')&(vis['Alt Coins Currency'].notnull())]
visdata = visdata[['Kind','Amount Deposited USD','Actual Amount USD','Alt Coins Amount','Alt Coins Currency','Complete Timestamp']]
visbuy = visdata[visdata['Kind']=='BUY']
visbuy = visbuy.groupby(['Alt Coins Currency','Complete Timestamp'],as_index=False).agg({'Amount Deposited USD':sum,'Kind':np.size})
visbuy = visbuy.rename(columns={'Alt Coins Currency': 'currency','Amount Deposited USD': 'dollars','Kind':'count','Complete Timestamp':'date'})
visbuy = visbuy.assign(type='buy')
visbuy = visbuy.assign(os='vis')
vissell = visdata[visdata['Kind']=='SELL']
vissell = vissell.groupby(['Alt Coins Currency','Complete Timestamp'],as_index=False).agg({'Actual Amount USD':sum,'Kind':np.size})
vissell = vissell.rename(columns={'Alt Coins Currency': 'currency','Actual Amount USD': 'dollars','Kind':'count','Complete Timestamp':'date'})
vissell = vissell.assign(type='buy')
vissell = vissell.assign(os='vis')
print(vissell.head())

  currency                     date  dollars  count type   os
0      BCH  04/01/2021 20:06:37 UTC    150.0      1  buy  vis
1      BCH  12/22/2021 11:30:11 UTC     80.0      1  buy  vis
2      BCH  12/22/2021 11:39:51 UTC    100.0      1  buy  vis
3      BCH  12/22/2021 20:37:29 UTC    220.0      1  buy  vis
4     DOGE  12/01/2021 17:34:29 UTC   1000.0      1  buy  vis


In [ ]:
vissell['date'] = pd.to_datetime(vissell['date']).dt.date
visbuy['date'] = pd.to_datetime(visbuy['date']).dt.date
basell['date'] = pd.to_datetime(basell['date']).dt.date
babuy['date'] = pd.to_datetime(babuy['date']).dt.date

table = pd.concat([babuy,basell,visbuy,vissell])
table = table.groupby(['currency','os','type','date'],as_index=False).agg({'dollars':sum,'count':sum})



In [ ]:
table.to_csv('Yan Request.csv',index=False)
files.download('Yan Request.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>